In [ ]:
!pip install segment-geospatial groundingdino-py leafmap localtileserver -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 15.1 MB/

In [ ]:
import geopandas as gpd
from samgeo import tms_to_geotiff
from samgeo.text_sam import LangSAM
import os
import leafmap

In [ ]:
# === Étape 1 : Charger depuis le KML ===
kml_file = "Projet_piscine.kml"
piscines_reelles = gpd.read_file(kml_file, driver="KML")


In [ ]:
# S'assurer que le CRS est bon
if piscines_reelles.crs is None:
    piscines_reelles.set_crs(epsg=4326, inplace=True)

print("Piscines détectées dans le fichier KML :", piscines_reelles.shape[0])


Piscines détectées dans le fichier KML : 3


In [ ]:
# === Étape 2 : Paramètres ===
prompts = ["swimming pool", "rectangular pool", "blue water", "public pool"]
sam = LangSAM()

# Marge autour de chaque point (en degrés ~0.0005 ≈ 50m)
margin = 0.0005

# Pour visualisation
m = leafmap.Map(center=[-21.4485, 47.087], zoom=17)
m.add_basemap("SATELLITE")
# Ajouter la vérité terrain
m.add_vector(kml_file, layer_name="Piscines réelles", style={"color": "green", "fillColor": "green", "fillOpacity": 0.5})


GroundingDINO_SwinB.cfg.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

groundingdino_swinb_cogcoor.pth:   0%|          | 0.00/938M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth" to /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.39G/2.39G [00:33<00:00, 75.9MB/s]


In [ ]:
# === Étape 3 : Boucle sur les piscines ===
for idx, row in piscines_reelles.iterrows():
    name = row["Name"].replace(" ", "_")
    point = row.geometry

    lon = point.x
    lat = point.y

    bbox = [lon - margin, lat - margin, lon + margin, lat + margin]
    image_path = f"Piscine_{name}.tif"

    print(f"\n📍 Traitement : {name} ({lat}, {lon})")

    # Télécharger l'image satellite
    tms_to_geotiff(output=image_path, bbox=bbox, zoom=19, source="Satellite", overwrite=True)

    # Appliquer plusieurs prompts
    for prompt in prompts:
        print(f"  🔍 Prompt : {prompt}")
        sam.predict(image_path, text_prompt=prompt, box_threshold=0.3, text_threshold=0.3)

    # Sauvegarder le masque
    mask_path = f"mask_{name}.tif"
    sam.show_anns(
        cmap="Blues",
        add_boxes=False,
        alpha=1,
        title=f"Détection : {name}",
        blend=False,
        output=mask_path,
    )

    # Exporter en vecteur
    shapefile_path = f"detect_{name}.shp"
    sam.raster_to_vector(mask_path, shapefile_path)

    # Ajouter au Leafmap
    m.add_vector(shapefile_path, layer_name=f"IA - {name}", style={"color": "blue", "fillColor": "blue", "fillOpacity": 0.4})



📍 Traitement : Piscine (-21.44825254264845, 47.08243778180142)
Downloaded image 1/6
Downloaded image 2/6
Downloaded image 3/6
Downloaded image 4/6
Downloaded image 5/6
Downloaded image 6/6
Saving GeoTIFF. Please wait...
Image saved to Piscine_Piscine.tif
  🔍 Prompt : swimming pool
  🔍 Prompt : rectangular pool
  🔍 Prompt : blue water
  🔍 Prompt : public pool


In [12]:
# === Étape 4 : Affichage final ===
m

Map(bottom=75299338.0, center=[-21.448510344436606, 47.08352744855685], controls=(ZoomControl(options=['positi…